In [1]:
# Import pandas library using an alias
import pandas as pd
# library to handle data in a vectorized manner
import numpy as np

# library to handle JSON files
# import json
# library to handle requests
# import requests
# tranform JSON file into a pandas dataframe
# from pandas.io.json import json_normalize

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# map rendering library
import folium

In [2]:
#library_df = pd.read_excel('ChicagoLibraries.xlsx')
#library_df.head()
ward49_df = pd.read_excel('Ward49.xlsx')
ward49_df.head()

,name,address,city,state,zip,phone,hours_of_operation,latitude,longitude,url,loc_type_pri,loc_type_sec
0,Rogers Park,6907 N. Clark St.,Chicago,IL,60626.0,(312) 744-0156,"Sun., Closed; Mon. & Wed., Noon-8; Tue. & Thu....",42.006708,-87.673408,https://www.chipublib.org/locations/61/,Library,Library
1,Sullivan Hs,6631 N Bosworth Ave,NaN,NaN,NaN,NaN,NaN,42.002688,-87.669192,NaN,School,High School
2,Gale,1631 W Jonquil Ter,NaN,NaN,NaN,NaN,NaN,42.021091,-87.671939,NaN,School,Elementary School
3,Acero - De La Cruz,7416 N Ridge Blvd,NaN,NaN,NaN,NaN,NaN,42.016476,-87.684406,NaN,School,Elementary School
4,Field,7019 N Ashland Ave,NaN,NaN,NaN,NaN,NaN,42.009664,-87.669985,NaN,School,Elementary School


In [3]:
ward49_df.dtypes

name                   object
address                object
city                   object
state                  object
zip                   float64
phone                  object
hours_of_operation     object
latitude              float64
longitude             float64
url                    object
loc_type_pri           object
loc_type_sec           object
dtype: object

In [4]:
#library_df.columns
ward49_df.columns

Index(['name', 'address', 'city', 'state', 'zip', 'phone',
       'hours_of_operation', 'latitude', 'longitude', 'url', 'loc_type_pri',
       'loc_type_sec'],
      dtype='object')

In [5]:
# get list of location types
location_type_list = ward49_df['loc_type_sec'].drop_duplicates().to_list()
location_type_list

['Library',
 'High School',
 'Elementary School',
 'Playground',
 'Beach',
 'Gymnasium',
 'Basketball Court',
 'Football/Soccer Combo Fld',
 'Baseball Jr/Softball',
 'Baseball Sr',
 'Tennis Court',
 'Boat Launch Non-Motorized',
 'Shuffleboard',
 'Boxing Center',
 'Volleyball',
 'Handball/Racquet (In)',
 'Dog Friendly Area',
 'Artificial Turf Field',
 'Nature/Bird Sanctuary',
 'Fitness Center',
 'Spray Feature',
 'Community Garden',
 'Fire Station']

In [6]:
# Get coordinates for Chicago
#address = 'Chicago, IL'
address = 'Rogers Park, Chicago, IL'
geolocator = Nominatim(user_agent='chi_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinates of Rogers Park, Chicago, IL are {}, {}".format(latitude, longitude))

The geographical coordinates of Rogers Park, Chicago, IL are 42.0090367, -87.6768492


## Create fixed map

In [7]:
# Create a map
chicago_map = folium.Map(location=[latitude, longitude],
                         zoom_start=14)

In [8]:
# Add markers to the map
for lat, lon, name in zip(ward49_df['latitude'],
                          ward49_df['longitude'],
                          ward49_df['name']):
    label = folium.Popup(str(name), parse_html=True)
    folium.CircleMarker(
       [lat, lon],
       radius=5,
       popup=label,
       fill=True,
       fill_opacity=0.7).add_to(chicago_map)

In [9]:
# Display the map
chicago_map

## Test interactive map

In [10]:
# Create a map
chicago_map = folium.Map(location=[latitude, longitude],
                         zoom_start=14)

In [11]:
#ward49_df[ward49_df['location_type'] == 'Library']
ward49_df[ward49_df['loc_type_pri'] == 'Library'][['latitude','longitude']]

,latitude,longitude
0,42.006708,-87.673408


In [12]:
points1 = [(42.006708, -87.673408)]

library_group = folium.FeatureGroup(name='Library').add_to(chicago_map)

for tuple_ in points1:
    label = folium.Popup('Rogers Park Library', parse_html=True)
    library_group.add_child(folium.CircleMarker(
        (42.006708, -87.673408),
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7)
    )

In [13]:
ward49_df[ward49_df['loc_type_sec'] == 'High School'][['latitude','longitude']]

,latitude,longitude
1,42.002688,-87.669192
6,42.013031,-87.674818


In [14]:
points2 = [(42.002688, -87.669192),
           (42.013031, -87.674818)]

hs_group = folium.FeatureGroup(name='High School').add_to(chicago_map)

for tuple_ in points2:
    label = folium.Popup('High School', parse_html=True)
    hs_group.add_child(folium.CircleMarker(
        tuple_,
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7)
    )

In [15]:
folium.LayerControl().add_to(chicago_map)
chicago_map

## Create Interactive Map

In [16]:
# Create a map
chicago_map = folium.Map(location=[latitude, longitude],
                         zoom_start=14)

In [17]:
fgroup = []

In [18]:
for i in range(len(location_type_list)):
    #print(i, location_type_list[i])
    fgroup.append(folium.FeatureGroup(name=location_type_list[i]).add_to(chicago_map))
    for row in ward49_df[ward49_df['loc_type_sec'] == location_type_list[i]].itertuples():
        #print(row.name, row.latitude, row.longitude)
        label = folium.Popup(row.name, parse_html=True)
        fgroup[i].add_child(folium.CircleMarker(
            [row.latitude, row.longitude],
            radius=5,
            popup=label,
            fill=True,
            fill_opacity=0.7).add_to(chicago_map)
        )
    
fgroup

In [19]:
folium.LayerControl().add_to(chicago_map)
chicago_map

## Try different marker types

In [20]:
# Create a map
chicago_map = folium.Map(location=[latitude, longitude],
                         zoom_start=14)

In [21]:
# Add markers to the map
for lat, lon, name in zip(ward49_df['latitude'],
                          ward49_df['longitude'],
                          ward49_df['name']):
    label = folium.Popup(str(name), parse_html=True)
    icon = folium.Icon(color='red', icon='bell', prefix='fa')
    folium.Marker(
       location=[lat, lon],
       icon=icon,
       popup=label).add_to(chicago_map)
    
chicago_map

## Customize interactive map

In [22]:
# Create a map
chicago_map = folium.Map(location=[latitude, longitude],
                         zoom_start=13)

fgroup = []
for i in range(len(location_type_list)):
    #print(i, location_type_list[i])
    fgroup.append(folium.FeatureGroup(name=location_type_list[i]).add_to(chicago_map))
    
    for row in ward49_df[ward49_df['loc_type_sec'] == location_type_list[i]].itertuples():
        #print(row.name, row.latitude, row.longitude)
        label = folium.Popup(row.name, parse_html=True)
        if row.loc_type_sec == 'Elementary School':
            icon = folium.Icon(color='red', icon='bell', prefix='fa')
        elif row.loc_type_sec == 'High School':
            icon = folium.Icon(color='blue', icon='graduation-cap', prefix='fa')
        elif row.loc_type_sec == 'Library':
            icon = folium.Icon(color='green', icon='book', prefix='fa')
        else:
            icon = folium.Icon(color='gray', icon='question-circle', prefix='fa')

        fgroup[i].add_child(folium.Marker(
            location=[row.latitude, row.longitude],
            icon=icon,
            popup=label).add_to(chicago_map)
        )
        
folium.LayerControl().add_to(chicago_map)
chicago_map

Not enough icons available to use them. Going back to plain circles.

## Interactive Map (take 2)

In [23]:
# Create a map
chicago_map = folium.Map(location=[latitude, longitude],
                         zoom_start=13)

fgroup = []
for i in range(len(location_type_list)):
    fgroup.append(folium.FeatureGroup(name=location_type_list[i],
                                      overlay=True,
                                      show=False).add_to(chicago_map))
    for row in ward49_df[ward49_df['loc_type_sec'] == location_type_list[i]].itertuples():
        if row.loc_type_pri == "Park":
            cir_color = "green"
        elif row.loc_type_pri == "Police":
            cir_color = "blue"
        elif row.loc_type_pri == "Fire Station":
            cir_color = "red"
        elif row.loc_type_pri == "School":
            cir_color = "orange"
        elif row.loc_type_pri == "Library":
            cir_color = "yellow"
        else:
            cir_color = "grey"
            
        tooltip_label = row.name + " " + row.loc_type_sec
        #popup_label = row.name + " " + row.loc_type_sec + " " + row.url
        popup_label = "{} {} {} {}".format(row.name, row.loc_type_sec, row.address, row.url)
            
        label = folium.Popup(html=popup_label, parse_html=True)
        fgroup[i].add_child(folium.CircleMarker(
            [row.latitude, row.longitude],
            radius=5,
            popup=label,
            tooltip=tooltip_label,
            color=cir_color,
            fill=True,
            fill_opacity=0.7).add_to(chicago_map)
        )
        
folium.LayerControl().add_to(chicago_map)
chicago_map